In [19]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

import pickle


In [20]:
# Device configuration

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)

device: cuda


In [21]:
from IPython.display import HTML, display


# Custom IPython progress bar for training
class ProgressMonitor(object):
    # 학습 과정의 progress를 추적하여 현재 loss value와 progress bar를 display

    tmpl = """
        <table style="width: 100%;">
            <tbody>
                <tr>
                    <td style="width: 30%;">
                     <b>Loss: {loss:0.4f}</b> &nbsp&nbsp&nbsp {value} / {length}
                    </td>
                    <td style="width: 70%;">
                        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
                    </td>
                </tr>
            </tbody>
        </table>
        """

    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)

    def html(self, count, loss):
        return HTML(self.tmpl.format(length=self.length, value=count, loss=loss))

    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

In [22]:
# CIFAR-10 dataset의 data transformation (data preprocessing)

transform_train = T.Compose( [T.RandomCrop(32, padding=4), T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )
'''
T.RandomCrop(32, padding = 4)
- input image : 32 * 32 size + 4 padding pixel
- training data에 diversity 주는 data augmentation 기술 (본 이미지로부터 랜덤하게 패치 추출)
'''
# data augmentation : 32 * 32 size에 padding 4로 랜덤하게 이미지 crop
# tensor conversion & scaling
# tensor normalization : subtract 0.5 & divide 0.5 for each channel
transform_test = T.Compose( [T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )
'''
T.ToTensor()
- input image : NumPy array , range [0, 255]
- output : PyTorch tensor, range [0, 1]
'''
# tensor conversion & scaling
# tensor normalization : subtract 0.5 & divide 0.5 for each channel

train_set = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train )
# CIFAR-10 학습데이터에 대한 인스턴스 생성
# transform_train transformation 수행

test_set = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test )
# CIFAR-10 test data에 대한 인스턴스 생성
# transform_test transformation 수행


classes = train_set.classes     # CIFAR-10 dataset에 대한 클래스 레이블 저장

Files already downloaded and verified
Files already downloaded and verified


In [23]:
print(train_set.data.shape)
print(test_set.data.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [24]:
# 과제 1- SimpleCNN의 오류를 없애라!
# Hint- matrix의 size를 주의! maxpooling은 size를 1/2배한다. filter의 size도 중요

class SimpleCNN(nn.Module):

    def __init__(self):
        super().__init__()

        #### 입력 이미지 크기 32x32x3
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d(2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.MaxPool2d(2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 8 * 8, 500),  # 마지막 convolutional layer에 맞게 flattened size 맞추기

            nn.Linear(500, 10),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

In [25]:
# 모델 테스트
# 텐서의 사이즈가 (7, 10)이 나오면 성공
# 현재는 오류가 뜨는 상황! matrix size를 잘 맞춰서 이 코드가 정상적으로 구동되면 성공입니다.

temp = SimpleCNN()
output = torch.randn( 7, 3, 32, 32)

print( temp(output).size() )


torch.Size([7, 10])


## ResNet
- 개요 : skip-connection을 통해 Residual을 학습할 수 있다
- implementation
    - Resblock : 1 * 1 convolution으로 channel depth를 줄이는 bottleneck 이후 3 * 3 convolution을 실시하고, 다시 1 * 1 convolution을 취함
    

In [34]:
# 실습 2- Resnet-18 구현하기
# Hint- layer를 지나간 뒤 input을 더해주어야 한다, stride말고 maxpool로 size 줄여도 괜찮습니다.

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride = 1):
        super(BasicBlock, self).__init__()

        # stride 통해 높이와 너비 조정
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace = True)

        # stride = 1, padding = 1이므로 높이와 너비 유지
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # x 그대로 더해줌
        self.shortcut = nn.Sequential()

        # 만약 size 안 맞아서 합 연산 불가할 경우, 연산 가능하도록 모양 맞춰줌
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)     # 필요에 따라 layer skip
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes = 10):
        super(ResNet, self).__init__()
        # RGB 3개의 channel, 64개의 kernel 사용
        self.in_channels = 64
        # ResNet Model의 Layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self.make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    # 다양한 architecture 생성 위해 make_layer로 sequential 생성
    def make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [35]:
# Resnet 모델 테스트
# 텐서의 사이즈가 (7, 10)이 나오면 성공

temp = ResNet18()   # ResNet18 model 인스턴스 생성
output = torch.randn(7, 3, 32, 32)  # (batch_size, channels, height, width)

print(temp(output).size())  # forward pass


torch.Size([7, 10])


In [36]:
batch_size = 128 # 배치 사이즈
learning_rate = 0.01 # 학습률
num_epochs = 30 # 에폭 수

In [37]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [38]:
# 원하는 모델을 돌려보세요

model = ResNet18()

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1

In [39]:
# Loss Function
criterion = nn.CrossEntropyLoss()
# optimizer 선정
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [40]:
from statistics import mean

def train(optimizer, model, num_epochs=10, first_epoch=1):

    criterion = nn.CrossEntropyLoss()

    train_losses = []
    test_losses = []

    for epoch in range(first_epoch, first_epoch + num_epochs):
        print('Epoch', epoch)

        # train phase
        model.train()

        # create a progress bar
        progress = ProgressMonitor(length=len(train_set))

        # keep track of predictions
        correct_train = 0

        batch_losses = []

        for batch, targets in train_loader:

            # Move the training data to the GPU
            batch = batch.to(device)
            targets = targets.to(device)

            # clear previous gradient computation
            optimizer.zero_grad()

            # forward propagation
            outputs = model(batch)

            # calculate the loss
            loss = criterion(outputs, targets)

            # backpropagate to compute gradients
            loss.backward()

            # update model weights
            optimizer.step()

            batch_losses.append(loss.item())

            # accumulate correct count
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == targets.data)

            # update progress bar
            progress.update(batch.shape[0], mean(batch_losses) )


        train_losses.append( mean(batch_losses))


        # test phase
        model.eval()

        y_pred = []

        correct_test = 0

        # We don't need gradients for test, so wrap in
        # no_grad to save memory
        with torch.no_grad():

            for batch, targets in test_loader:

                # Move the training batch to the GPU
                batch = batch.to(device)
                targets = targets.to(device)

                # forward propagation
                outputs = model(batch)

                # calculate the loss
                loss = criterion(outputs, targets)

                # save predictions
                y_pred.extend( outputs.argmax(dim=1).cpu().numpy() )

                # accumulate correct count
                _, preds = torch.max(outputs, 1)
                correct_test += torch.sum(preds == targets.data)


        # Calculate accuracy
        train_acc = correct_train.item() / train_set.data.shape[0]
        test_acc = correct_test.item() / test_set.data.shape[0]

        print('Training accuracy: {:.2f}%'.format(float(train_acc) * 100))
        print('Test accuracy: {:.2f}%\n'.format(float(test_acc) * 100))


    return train_losses, test_losses, y_pred

In [41]:
#@title
train_losses, test_losses, y_pred = train(optimizer, model, num_epochs=num_epochs)

Epoch 1


Loss: 1.8528 50000 / 50000,50000


Training accuracy: 32.04%
Test accuracy: 39.70%

Epoch 2


Loss: 1.3852 50000 / 50000,50000


Training accuracy: 48.56%
Test accuracy: 52.86%

Epoch 3


...

Training accuracy: 97.35%
Test accuracy: 88.70%

Epoch 29


Loss: 0.0692 50000 / 50000,50000


Training accuracy: 97.60%
Test accuracy: 88.04%

Epoch 30


Loss: 0.0658 50000 / 50000,50000


Training accuracy: 97.78%
Test accuracy: 88.26%



여기까지 진행하고! ipynb 파일 제출해주세요.